# Order Sorting Recipe
v2019-05-07.01 - draft  
v2019-05-08.01 - add prediction


In [1]:
# setup for training AND prediction
import analitico
import analitico.plugin
import s24.plugin

from analitico.pandas import *

import numpy as np
from scipy import stats

# pass api token to create factory
factory = analitico.authorize("tok_s24_579E5hOWw7k8")

import datetime
print(datetime.datetime.utcnow())

import logging
logger = logging.getLogger("s24")

2019-05-08 13:43:54.767779


In [2]:
# data used for sample prediction
data = [
		{
			"ord_id": 980188.0,
			"sto_ref_id": 326.0,
			"sto_name": "carrefour market",
			"sto_area": "RM8",
            "sto_province": "RM",
			"details": [
				{
					"odt_id": 19415192,
					"odt_ean": 8024666500072.0,
					"odt_name": "Carrefour Bio Limoni bio",
					"odt_category_id": 251
				},
				{
					"odt_id": 19415196,
					"odt_ean": 2002146130003.0,
					"odt_name": "Ananas ",
					"odt_category_id": 254
				},
				{
					"odt_id": 19415194,
					"odt_ean": 2002054130003.0,
					"odt_name": "Peperoni Rossi ",
					"odt_category_id": 261
				},
				{
					"odt_id": 19415198,
					"odt_ean": 2002322130001.0,
					"odt_name": "Peperoni Gialli ",
					"odt_category_id": 261
				},
				{
					"odt_id": 19415180,
					"odt_ean": 8004381107008.0,
					"odt_name": "Mele Alto Adige Igp Golden Delicious Italia Cal. i 80-85 Mm 0,900 Kg",
					"odt_category_id": 252
				},
				{
					"odt_id": 19415200,
					"odt_ean": 2002135130007.0,
					"odt_name": "Arance Navelinas ",
					"odt_category_id": 251
				},
				{
					"odt_id": 19415188,
					"odt_ean": 8032603412043.0,
					"odt_name": "Carrefour Pomodoro Datterino 250 Gr.",
					"odt_category_id": 260
				},
				{
					"odt_id": 19415176,
					"odt_ean": 8013165060057.0,
					"odt_name": "Zorbas Light Formaggio Greco Magro",
					"odt_category_id": 286
				},
				{
					"odt_id": 19415190,
					"odt_ean": 3245414229228.0,
					"odt_name": "Carrefour Formaggio Fresco di Capra",
					"odt_category_id": 298
				},
				{
					"odt_id": 19415184,
					"odt_ean": 8010292009874.0,
					"odt_name": "Insalata belga  ",
					"odt_category_id": 264
				},
				{
					"odt_id": 19415178,
					"odt_ean": 8033661091003.0,
					"odt_name": "Amaltea Formaggio Bocconcino di Capra",
					"odt_category_id": 298
				},
				{
					"odt_id": 19415182,
					"odt_ean": 8001350020627.0,
					"odt_name": "Carrefour Bio Pomodori Pelati",
					"odt_category_id": 450
				},
				{
					"odt_id": 19415172,
					"odt_ean": 9001475062341.0,
					"odt_name": "Pompadour Zenzero Limone per Infuso",
					"odt_category_id": 422
				},
				{
					"odt_id": 19415170,
					"odt_ean": 8007185002722.0,
					"odt_name": "Everton Karkadè, Infuso Bio",
					"odt_category_id": 422
				},
				{
					"odt_id": 19415186,
					"odt_ean": 8012666041916.0,
					"odt_name": "Carrefour Bio Polpa di Pomodoro in Succo di Pomodoro",
					"odt_category_id": 451
				},
				{
					"odt_id": 19415168,
					"odt_ean": 8002280003506.0,
					"odt_name": "Gemma di Mare Gli Integrali Sale Marino del Mediterraneo Grosso",
					"odt_category_id": 6009
				},
				{
					"odt_id": 19415174,
					"odt_ean": 8020141810001.0,
					"odt_name": "Sant'Anna Naturale Sorgente Rebruant ",
					"odt_category_id": 691
				},
				{
					"odt_id": 19415174,
					"odt_ean": 8020141810001.0,
					"odt_name": "Sant'Anna Naturale Sorgente Rebruant ",
					"odt_category_id": -2
				}
			]
		}
	]

In [3]:
df1 = factory.run_plugin(action="train", settings = {
    "name": "analitico.plugin.DatasetSourcePlugin",
    "dataset_id": "ds_s24_outofstock_nb"
})

In [4]:
df1.sample(4)

,odt_id,odt_ean,odt_name,odt_category_id,odt_replaceable,odt_variable_weight,odt_price,odt_price_per_type,odt_surcharge_fixed,odt_touched_at,odt_status,ord_id,sto_id,sto_name,sto_area,sto_province,sto_ref_id
5632774,23314504,5201054015667,Fage Total 0% Grassi,6046,3,0,1.19,6.0,0.0,2019-04-10 10:21:55,PURCHASED,1144358,1127328,famila,PD1,PD,2415
5278820,22606058,920112424,Realforno Canestrelli,408,3,0,0.99,3.0,0.0,2019-03-28 18:01:33,PURCHASED,1115258,1098114,lidl,RM12,RM,5272
3397262,18842942,8012666051731,Carrefour Mandorle Tostate e Salate,552,3,0,1.99,19.0,0.0,2019-01-18 16:24:11,PURCHASED,955266,937546,carrefour market,MI2,MI,5218
4587911,21224240,920244194,"Freshona Misto per Soffritto Cipolle, Sedano, ...",59,3,0,0.75,2.0,0.0,2019-03-02 11:30:21,NOT_FOUND,1057388,1039970,lidl,MI5,MI,5231


In [5]:
records = len(df1.index)

# remove rows which have the string "nan" as odt_ean or odt_category_id
# notice how this is nan the string, not np.NaN the correct not a number
# constant. these are coming from encoding errors in the original source tables.
df1.drop(df1[df1["odt_category_id"] == "nan"].index, inplace=True)
df1.drop(df1[df1["odt_ean"] == "nan"].index, inplace=True)

# missing store province
df1.drop(df1[df1["sto_province"].isna()].index, inplace=True)

if len(df1.index) < records:
    logger.warning(f"{records - len(df1.index)} records with nan values where dropped")
    
# check to make sure we don't have null values
df1.isna().sum()

30042 records with nan values where dropped


odt_id                       0
odt_ean                      0
odt_name                     0
odt_category_id              0
odt_replaceable              0
odt_variable_weight          0
odt_price                    0
odt_price_per_type           0
odt_surcharge_fixed          0
odt_touched_at         1592825
odt_status                   0
ord_id                       0
sto_id                       0
sto_name                     0
sto_area                     0
sto_province                 0
sto_ref_id                   0
dtype: int64

In [6]:
# expand category IDs
df2 = factory.run_plugin(df1, action="train", settings = {
    "name": "s24.plugin.AugmentCategoriesPlugin",
    "schema": {
        "columns": [
            { "name": "odt_category_id" }
        ]}
    })

In [7]:
# train model which will guess distance in seconds between tomatoes and milk
training = factory.run_plugin(df2, action="train", settings = {
    "name": "s24.plugin.OrderSortingPlugin",
        "parameters": {
          "iterations": 30,
          "learning_rate": 0.50
        },
        "data": {
          "chronological": True,
          "label": "dyn_elapsed_sec"
        }
    })

0:	learn: 56.1214414	test: 53.7491066	best: 53.7491066 (0)	total: 756ms	remaining: 21.9s
1:	learn: 49.7050959	test: 47.6921815	best: 47.6921815 (1)	total: 1.23s	remaining: 17.2s
2:	learn: 47.7315386	test: 45.9512808	best: 45.9512808 (2)	total: 1.83s	remaining: 16.5s
3:	learn: 47.1609385	test: 45.5303875	best: 45.5303875 (3)	total: 2.33s	remaining: 15.1s
4:	learn: 46.9141757	test: 45.3864698	best: 45.3864698 (4)	total: 2.81s	remaining: 14.1s
5:	learn: 46.7523140	test: 45.2865916	best: 45.2865916 (5)	total: 3.13s	remaining: 12.5s
6:	learn: 46.4018109	test: 44.9769476	best: 44.9769476 (6)	total: 3.87s	remaining: 12.7s
7:	learn: 46.3510008	test: 44.9436029	best: 44.9436029 (7)	total: 4.37s	remaining: 12s
8:	learn: 46.2702802	test: 44.8779191	best: 44.8779191 (8)	total: 5.04s	remaining: 11.8s
9:	learn: 46.0733560	test: 44.6797633	best: 44.6797633 (9)	total: 5.73s	remaining: 11.5s
10:	learn: 45.9035006	test: 44.5333213	best: 44.5333213 (10)	total: 6.4s	remaining: 11.1s
11:	learn: 45.7835229	

In [8]:
training

OrderedDict([('type', 'analitico/training'),
             ('plugins',
              {'training': 's24.plugin.OrderSortingPlugin',
               'prediction': 's24.plugin.OrderSortingPlugin'}),
             ('data',
              {'label': 'dyn_elapsed_sec',
               'chronological': True,
               'schema': {'columns': [{'name': 'odt_status',
                  'type': 'category'},
                 {'name': 'sto_ref_id', 'type': 'category'},
                 {'name': 'sto_name', 'type': 'category'},
                 {'name': 'sto_area', 'type': 'category'},
                 {'name': 'sto_province', 'type': 'category'},
                 {'name': 'prev_odt_category_id', 'type': 'integer'},
                 {'name': 'prev_odt_category_id.level2', 'type': 'integer'},
                 {'name': 'prev_odt_category_id.level3', 'type': 'integer'},
                 {'name': 'odt_category_id', 'type': 'integer'},
                 {'name': 'odt_category_id.level2', 'type': 'integer'},


In [9]:
training["scores"]["features_importance"]

{'odt_category_id': 41.15586,
 'prev_odt_category_id': 27.78996,
 'odt_status': 12.95166,
 'sto_name': 10.97179,
 'sto_area': 3.32937,
 'sto_ref_id': 2.04537,
 'sto_province': 1.75599,
 'prev_odt_category_id.level2': 0.0,
 'prev_odt_category_id.level3': 0.0,
 'odt_category_id.level2': 0.0,
 'odt_category_id.level3': 0.0}

In [10]:
# generate predictions
results = factory.run_plugin(data, action="predict", settings = {
        "name": "s24.plugin.OrderSortingPlugin"
    }
)

item, cat_id


In [11]:
results

OrderedDict([('type', 'analitico/prediction'),
             ('performance',
              {'cpu_count': 12,
               'loading_ms': 0,
               'items': 18,
               'predictions': 342,
               'predictions_ms': 354,
               'total_ms': 490}),
             ('records',
              [{'ord_id': 980188.0,
                'sto_ref_id': 326.0,
                'sto_name': 'carrefour market',
                'sto_area': 'RM8',
                'sto_province': 'RM',
                'details': [{'odt_id': 19415192,
                  'odt_ean': 8024666500072.0,
                  'odt_name': 'Carrefour Bio Limoni\xa0bio',
                  'odt_category_id': 251},
                 {'odt_id': 19415196,
                  'odt_ean': 2002146130003.0,
                  'odt_name': 'Ananas ',
                  'odt_category_id': 254},
                 {'odt_id': 19415194,
                  'odt_ean': 2002054130003.0,
                  'odt_name': 'Peperoni Rossi ',
      